In [1]:
%reset -f

In [22]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from os.path import join

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"

In [4]:
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

In [5]:
msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]

,subject_id,subject_label,predicate_id,object_id,mapping_justification
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified
...,...,...,...,...,...
71865,MONDO:0100347,carcinoid syndrome,skos:exactMatch,ICD10CM:E34.0,Unspecified
71939,MONDO:0100471,vitamin D deficiency,skos:exactMatch,ICD10CM:E55,Unspecified
71954,MONDO:0100491,generalized pustular psoriasis,skos:exactMatch,ICD10CM:L40.1,Unspecified
72050,MONDO:0800029,interstitial lung disease 2,skos:exactMatch,ICD10CM:J84.112,Unspecified


In [6]:
msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
67,DOID:0050052,Rocky Mountain spotted fever,skos:exactMatch,ICD10CM:A77.0,Spotted fever due to Rickettsia rickettsii,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,Rocky Mountain spotted fever
73,DOID:0050083,Keshan disease,skos:exactMatch,ICD10CM:E59,Dietary selenium deficiency,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,Keshan disease
83,DOID:0050118,La Crosse encephalitis,skos:exactMatch,ICD10CM:A83.5,California encephalitis,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,La Crosse encephalitis
158,DOID:0050195,Bolivian hemorrhagic fever,skos:exactMatch,ICD10CM:A96.1,Machupo hemorrhagic fever,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,Bolivian hemorrhagic fever
163,DOID:0050200,Korean hemorrhagic fever,skos:exactMatch,ICD10CM:A98.5,Hemorrhagic fever with renal syndrome,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,Korean hemorrhagic fever
...,...,...,...,...,...,...,...,...,...,...,...
37509,ICD10WHO:Y61.5,During heart catheterization,skos:exactMatch,ICD10WHO:Y62.5,During heart catheterization,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,During heart catheterization
37510,ICD10WHO:Y61.6,"During aspiration, puncture and other catheter...",skos:exactMatch,ICD10WHO:Y62.6,"During aspiration, puncture and other catheter...",semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,"During aspiration, puncture and other catheter..."
37511,ICD10WHO:Y61.8,During other surgical and medical care,skos:exactMatch,ICD10WHO:Y62.8,During other surgical and medical care,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,During other surgical and medical care
37512,ICD10WHO:Y61.9,During unspecified surgical and medical care,skos:exactMatch,ICD10WHO:Y62.9,During unspecified surgical and medical care,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,During unspecified surgical and medical care


In [7]:
msdf_lex.prefix_map

{'DOID': 'http://purl.obolibrary.org/obo/DOID_',
 'ICD10CM': 'http://purl.bioontology.org/ontology/ICD10CM/',
 'ICD10WHO': 'https://icd.who.int/browse10/2019/en#/',
 'MONDO': 'http://purl.obolibrary.org/obo/MONDO_',
 'NCIT': 'http://purl.obolibrary.org/obo/NCIT_',
 'skos': 'http://www.w3.org/2004/02/skos/core#',
 'owl': 'http://www.w3.org/2002/07/owl#',
 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
 'semapv': 'https://w3id.org/semapv/',
 'sssom': 'https://w3id.org/sssom/'}

In [8]:
msdf_mondo.metadata

{'license': 'https://w3id.org/sssom/license/unspecified',
 'mapping_set_id': 'https://w3id.org/sssom/mappings/9afc4b25-a436-451a-b35d-b87eb2e2fed5'}

In [9]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe

In [10]:
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains('UNIQUE')) & 
    (comparison_df['object_id'].str.startswith("ICD"))
]

unmapped_df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment,object_label,mapping_tool,confidence,subject_match_field,object_match_field,match_string
2401,MONDO:0001195,spotted fever,skos:broadMatch,ICD10CM:A77.0,Unspecified,UNIQUE_1,NaN,NaN,NaN,NaN,NaN,NaN
58763,MONDO:0018381,osteochondrosis,skos:exactMatch,ICD10CM:M42,Unspecified,UNIQUE_1,NaN,NaN,NaN,NaN,NaN,NaN
52404,MONDO:0015901,obsolete rare inherited hyperlipidemia,skos:broadMatch,ICD10CM:E78.1,Unspecified,UNIQUE_1,NaN,NaN,NaN,NaN,NaN,NaN
17283,MONDO:0005885,optic neuritis,skos:exactMatch,ICD10CM:H46,Unspecified,UNIQUE_1,NaN,NaN,NaN,NaN,NaN,NaN
66080,MONDO:0021464,benign neoplasm of cecum,skos:exactMatch,ICD10CM:D12.0,Unspecified,UNIQUE_1,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame

ICD_prefix_map = {
    "ICD10CM": "http://purl.bioontology.org/ontology/ICD10CM/",
    "ICD10WHO": "https://icd.who.int/browse10/2019/en#/"
}
msdf_lex.prefix_map.update(ICD_prefix_map)
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)


In [12]:
df_dict = split_dataframe(combined_msdf)

In [20]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

['icd10cm_closematch_mondo',
 'icd10cm_exactmatch_mondo',
 'icd10cm_broadmatch_mondo',
 'icd10cm_narrowmatch_mondo',
 'mondo_exactmatch_icd10who',
 'mondo_exactmatch_icd10cm',
 'mondo_broadmatch_icd10cm',
 'icd10who_closematch_mondo',
 'icd10who_exactmatch_mondo',
 'icd10who_broadmatch_mondo',
 'icd10who_narrowmatch_mondo']

In [23]:
dir_name = "dataframes"
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [15]:
df_dict['mondo_broadmatch_icd10cm'].df

,subject_id,subject_label,predicate_id,object_id,mapping_justification,comment
0,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,UNIQUE_1
1,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,UNIQUE_1
2,MONDO:0000827,salmonellosis,skos:broadMatch,ICD10CM:A01.4,Unspecified,UNIQUE_1
3,MONDO:0000827,salmonellosis,skos:broadMatch,ICD10CM:A02.0,Unspecified,UNIQUE_1
4,MONDO:0000827,salmonellosis,skos:broadMatch,ICD10CM:A01.3,Unspecified,UNIQUE_1
...,...,...,...,...,...,...
880,MONDO:0020502,yellow fever,skos:broadMatch,ICD10CM:A95.1,Unspecified,UNIQUE_1
881,MONDO:0020502,yellow fever,skos:broadMatch,ICD10CM:A95.0,Unspecified,UNIQUE_1
882,MONDO:0020502,yellow fever,skos:broadMatch,ICD10CM:A95.9,Unspecified,UNIQUE_1
883,MONDO:0020540,ovarian gynandroblastoma,skos:broadMatch,ICD10CM:D40.1,Unspecified,UNIQUE_1
